In [ ]:
import os
import random
import numpy as np
import tensorflow as tf;
import matplotlib.pyplot as plt
from tqdm import *
from ocr_model import *
from image_utils import *
from random import randint
from training_set_utils import *
from PIL import Image, ImageEnhance, ImageOps

In [ ]:
model = OcrModel()
model.load_weights()

In [ ]:
%matplotlib inline
data_set = create_from_labeled_dir("images/labeled/")
data_set = letterbox_resize(data_set, 30, 42)
data_set = data_set + create_inverted_images(data_set)
data_set = equalize_classes(data_set)
data_set = create_brightness_variations(data_set, 0.6, 1.5)

In [ ]:
%matplotlib inline
#atlas_data_set = create_from_atlas_dir("images/atlas/", 24, 40)

In [ ]:
%matplotlib inline
#atlas_data_set = create_variations(atlas_data_set)
#data_set = data_set + atlas_data_set
#atlas_data_set = None

In [ ]:
%matplotlib inline

random.shuffle(data_set)

num_test_data = 1000
test_set = data_set[:num_test_data]
test_xy = model.convert_data_set(test_set)

train_set = data_set[num_test_data:]
train_xy = model.convert_data_set(train_set)

train_set = None
data_set = None

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import time
import math

use_generator = True
batch_size = 4 * 1024

gen = ImageDataGenerator(
        width_shift_range=3, 
        height_shift_range=3, 
        zoom_range=0.1, 
        horizontal_flip=False)
        #rotation_range=10,
        #brightness_range=[0.5, 1.0])

last = time.time()
for i in range(99999):

    #use_generator = i % 2 == 1

    if use_generator:        
        model.cnn.fit_generator(gen.flow(
            train_xy[0], train_xy[1],
            batch_size=batch_size, shuffle=True), epochs=50, workers=24, steps_per_epoch=math.ceil(len(train_xy[0]) / batch_size))
    else:
        model.cnn.fit(
            train_xy[0], train_xy[1],
            epochs=50,
            batch_size=batch_size)

    now = time.time()
    if now - last > 300:
        last = now
        loss_acc = model.cnn.evaluate(test_xy[0], test_xy[1],batch_size=batch_size)    
        model.save_weights("acc" + str(loss_acc[1]))

In [ ]:
def argmax2(arr):
    idx0 = 0
    last_max = 0.0
    for i in range(len(arr)):
        if arr[i] > last_max:
            last_max = arr[i]
            idx0 = i    
    idx1 = 0
    last_max = 0.0
    for i in range(len(arr)):
        if i != idx0 and arr[i] > last_max:
            last_max = arr[i]
            idx1 = i
    return (idx0, idx1)

def confidence(arr, pred):
    conv0 = arr[pred[0]]
    conv1 = arr[pred[1]]
    return (round(conv0, 2), round(conv1, 2))

def predict(img):
    result = model.predict(img)
    pred = argmax2(result)
    return (pred, confidence(result, pred))

def predict_test(idx):
    result = predict(test_set[idx].image)
    pred = result[0]
    conf = result[1]
    expect = argmax2(test_set[idx].y)
    expect_conf = confidence(test_set[idx].y, expect)
    if expect_conf[0] > 0.6:
        correct = expect[0] == pred[0]
        print("correct: " + str(correct) + 
              "; expected: " + str(expect) + " " + str(expect_conf) +
              "; predicted: " + str(pred) + " " + str(conf))
        if not correct: # or convidence < 0.9:
            test_set[idx].show()
            #plt.imshow(x_test[idx])
            #plt.show()
    else:
        print("skip")
for i in range(len(test_set)):
    predict_test(i)

In [ ]:
model.save_weights()

model.export_as_tflite()